### 1 背景
#### 有一批交易数据，数据总量达到28万多条。其中，正常交易数据量占比99.83%，欺诈交易数据量仅占比0.17%，此案例属于一个典型的不平衡数据案例。

### 2 目标
#### 训练出一个模型，能够判断出欺诈交易数据。

### 方案三：
#### 基于PyTorch的DL（深度学习）解决方法

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 加载数据文件
df = pd.read_csv("creditcard.csv")

In [3]:
# 默认显示前5行
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
# 数据表中的大多数列的数据已经归一化，接下来对Amount进行归一化
from sklearn.preprocessing import StandardScaler

df['scaled_amount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1,1)) # 金额归一化
df['scaled_time'] = StandardScaler().fit_transform(df['Time'].values.reshape(-1,1)) # 时间归一化

df.drop(['Amount', 'Time'], axis=1, inplace=True) # 删除原始的数据列

In [5]:
X = df.drop('Class', axis=1) # 数据集
y = df['Class'] # 标签

print("X shape : ", X.shape)
print("y shape : ", y.shape)

X shape :  (284807, 30)
y shape :  (284807,)


In [6]:
# 数据集分离：Train 和 Test
from sklearn.model_selection import train_test_split

X_new_train, X_new_test, y_new_train, y_new_test = train_test_split(X, y, test_size=0.1, random_state=666)

In [7]:
# 数据类型转换：DataFrame ———>array ————> Tensor

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optimizer

train = torch.utils.data.TensorDataset(torch.tensor(X_new_train.values).float(), 
                                         torch.tensor(y_new_train.values).float())

val = torch.utils.data.TensorDataset(torch.tensor(X_new_test.values).float(), 
                                       torch.tensor(y_new_test.values).float())

# 另一种写法
# X_train = torch.from_numpy(X_new_train.values)
# y_train = torch.from_numpy(y_new_train.values)
# train = torch.utils.data.TensorDataset(X_train, y_train)

# X_val = torch.from_numpy(X_new_test.values)
# y_val = torch.from_numpy(y_new_test.values)
# val = torch.utils.data.TensorDataset(X_val, y_val)

In [8]:
# 一批一批地读取数据
batch_size = 64

train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size)

val_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size)

In [9]:
# 自定义3层隐藏层的网络
class Model(torch.nn.Module):
    def __init__(self, input_dim=30, hidden_unit=16):
        super(Model, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_unit) # 输入: 30 , 有30个特征
        self.fc2 = torch.nn.Linear(hidden_unit, 18)
        self.fc3 = torch.nn.Linear(18, 20)
        self.relu = torch.nn.ReLU() 
        self.fc4 = torch.nn.Linear(20, 24)
        self.fc5 = torch.nn.Linear(24, 1) # 输出：1， 一个结果，是否欺诈
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = F.dropout(x, p=0.25)
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = F.sigmoid(self.fc5(x))
        return x

In [10]:
input_dim = X_new_train.shape[1] # 列数

hidden_unit = 16 # 16个隐藏单元

epochs = 10 # 运行多少轮

model = Model(input_dim, hidden_unit) # 创建模型

loss = torch.nn.BCELoss() # 损失函数

learning_rate = 0.003 # 学习率

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
# 定义训练函数
def train(epochs, model, loss_func, optimizer, train_loader):
    for epoch in range(epochs):
        model.train() # 模型训练
        for (i, train) in enumerate(train_loader):
            X, y = train # 获取训练数据和标签
            y_pred = model(X) # 预测
            loss = loss_func(y_pred, y) # 计算损失
            optimizer.zero_grad() # 重置梯度
            loss.backward() # 反向传播
            optimizer.step() # 更新参数
            print('epoch : {}, batch : {}, Loss : {}'.format(epoch, i, loss.item()) )# 输出损失

In [12]:
train(epochs, model, loss, optimizer, train_dataloader)

epoch : 0, batch : 0, Loss : 0.6325116157531738
epoch : 0, batch : 1, Loss : 0.621732234954834
epoch : 0, batch : 2, Loss : 0.6140957474708557
epoch : 0, batch : 3, Loss : 0.6008308529853821
epoch : 0, batch : 4, Loss : 0.5874056220054626
epoch : 0, batch : 5, Loss : 0.5754334926605225
epoch : 0, batch : 6, Loss : 0.5654159784317017
epoch : 0, batch : 7, Loss : 0.5495773553848267
epoch : 0, batch : 8, Loss : 0.5406009554862976
epoch : 0, batch : 9, Loss : 0.5201658606529236
epoch : 0, batch : 10, Loss : 0.5057559609413147
epoch : 0, batch : 11, Loss : 0.4923290014266968
epoch : 0, batch : 12, Loss : 0.4823603928089142
epoch : 0, batch : 13, Loss : 0.4617466330528259
epoch : 0, batch : 14, Loss : 0.4439607262611389
epoch : 0, batch : 15, Loss : 0.41313105821609497
epoch : 0, batch : 16, Loss : 0.38867953419685364
epoch : 0, batch : 17, Loss : 0.36709490418434143
epoch : 0, batch : 18, Loss : 0.34129855036735535
epoch : 0, batch : 19, Loss : 0.30245599150657654
epoch : 0, batch : 20, Los

epoch : 0, batch : 160, Loss : 0.0007069456041790545
epoch : 0, batch : 161, Loss : 0.00020144987502135336
epoch : 0, batch : 162, Loss : 0.0004966020351275802
epoch : 0, batch : 163, Loss : 0.00035568157909438014
epoch : 0, batch : 164, Loss : 0.0007747472845949233
epoch : 0, batch : 165, Loss : 0.0003718611260410398
epoch : 0, batch : 166, Loss : 0.0008540164562873542
epoch : 0, batch : 167, Loss : 0.00037502055056393147
epoch : 0, batch : 168, Loss : 0.00036178846494294703
epoch : 0, batch : 169, Loss : 0.0003085473144892603
epoch : 0, batch : 170, Loss : 0.00027592614060267806
epoch : 0, batch : 171, Loss : 0.000620061531662941
epoch : 0, batch : 172, Loss : 0.0004005709779448807
epoch : 0, batch : 173, Loss : 0.0003575580776669085
epoch : 0, batch : 174, Loss : 0.0006827859324403107
epoch : 0, batch : 175, Loss : 0.00043326671584509313
epoch : 0, batch : 176, Loss : 0.00024255986500065774
epoch : 0, batch : 177, Loss : 0.0005957433604635298
epoch : 0, batch : 178, Loss : 0.0003210

epoch : 0, batch : 323, Loss : 0.00043259430094622076
epoch : 0, batch : 324, Loss : 0.0007894391310401261
epoch : 0, batch : 325, Loss : 0.0005104158772155643
epoch : 0, batch : 326, Loss : 0.0346553660929203
epoch : 0, batch : 327, Loss : 0.00047575979260727763
epoch : 0, batch : 328, Loss : 0.0003156978636980057
epoch : 0, batch : 329, Loss : 0.0005122422007843852
epoch : 0, batch : 330, Loss : 0.03734590485692024
epoch : 0, batch : 331, Loss : 0.0005161018925718963
epoch : 0, batch : 332, Loss : 0.00038088863948360085
epoch : 0, batch : 333, Loss : 0.0006686507840640843
epoch : 0, batch : 334, Loss : 0.00029079243540763855
epoch : 0, batch : 335, Loss : 0.07253286242485046
epoch : 0, batch : 336, Loss : 0.00039592001121491194
epoch : 0, batch : 337, Loss : 0.0006509451195597649
epoch : 0, batch : 338, Loss : 0.00048142607556656003
epoch : 0, batch : 339, Loss : 0.0007413079729303718
epoch : 0, batch : 340, Loss : 0.00075620407005772
epoch : 0, batch : 341, Loss : 0.0005199479055590

epoch : 0, batch : 494, Loss : 0.00036702881334349513
epoch : 0, batch : 495, Loss : 0.000383848964702338
epoch : 0, batch : 496, Loss : 0.0001698563719401136
epoch : 0, batch : 497, Loss : 0.024807216599583626
epoch : 0, batch : 498, Loss : 0.00027770359884016216
epoch : 0, batch : 499, Loss : 0.0004304771719034761
epoch : 0, batch : 500, Loss : 0.0001801303878892213
epoch : 0, batch : 501, Loss : 0.0006910709198564291
epoch : 0, batch : 502, Loss : 0.000531114696059376
epoch : 0, batch : 503, Loss : 0.00025590663426555693
epoch : 0, batch : 504, Loss : 0.000244344788370654
epoch : 0, batch : 505, Loss : 0.00023095053620636463
epoch : 0, batch : 506, Loss : 0.00012274924665689468
epoch : 0, batch : 507, Loss : 0.00014439894584938884
epoch : 0, batch : 508, Loss : 0.046829260885715485
epoch : 0, batch : 509, Loss : 0.0007543474202975631
epoch : 0, batch : 510, Loss : 0.00019575681653805077
epoch : 0, batch : 511, Loss : 0.00011946052109124139
epoch : 0, batch : 512, Loss : 0.0008431791

epoch : 0, batch : 657, Loss : 0.00040989890112541616
epoch : 0, batch : 658, Loss : 0.0007559157675132155
epoch : 0, batch : 659, Loss : 0.021141890436410904
epoch : 0, batch : 660, Loss : 0.00067086738999933
epoch : 0, batch : 661, Loss : 0.00026536465156823397
epoch : 0, batch : 662, Loss : 0.03806008771061897
epoch : 0, batch : 663, Loss : 0.00031068676616996527
epoch : 0, batch : 664, Loss : 0.0013680282281711698
epoch : 0, batch : 665, Loss : 0.00020526449952740222
epoch : 0, batch : 666, Loss : 0.0013821213506162167
epoch : 0, batch : 667, Loss : 0.0007992961327545345
epoch : 0, batch : 668, Loss : 0.000294663681415841
epoch : 0, batch : 669, Loss : 0.00039208444650284946
epoch : 0, batch : 670, Loss : 0.00023913130280561745
epoch : 0, batch : 671, Loss : 0.0001331024250248447
epoch : 0, batch : 672, Loss : 0.00026348201208747923
epoch : 0, batch : 673, Loss : 0.00081398815382272
epoch : 0, batch : 674, Loss : 0.00045707746176049113
epoch : 0, batch : 675, Loss : 0.0001816347357

...

epoch : 9, batch : 3146, Loss : 0.00022420010645873845
epoch : 9, batch : 3147, Loss : 6.590179691556841e-05
epoch : 9, batch : 3148, Loss : 0.0006553531857207417
epoch : 9, batch : 3149, Loss : 0.00026536561199463904
epoch : 9, batch : 3150, Loss : 6.629768904531375e-05
epoch : 9, batch : 3151, Loss : 0.00010352431854698807
epoch : 9, batch : 3152, Loss : 9.338496602140367e-05
epoch : 9, batch : 3153, Loss : 2.3011252778815106e-05
epoch : 9, batch : 3154, Loss : 0.00015219568740576506
epoch : 9, batch : 3155, Loss : 4.1197912651114166e-05
epoch : 9, batch : 3156, Loss : 0.00018770612950902432
epoch : 9, batch : 3157, Loss : 0.002988917753100395
epoch : 9, batch : 3158, Loss : 0.0007930400897748768
epoch : 9, batch : 3159, Loss : 0.00014536864182446152
epoch : 9, batch : 3160, Loss : 0.00017031680908985436
epoch : 9, batch : 3161, Loss : 2.123267768183723e-05
epoch : 9, batch : 3162, Loss : 2.5302406356786378e-05
epoch : 9, batch : 3163, Loss : 0.00011683316552080214
epoch : 9, batch :

epoch : 9, batch : 3337, Loss : 0.0006220954819582403
epoch : 9, batch : 3338, Loss : 7.092892337823287e-05
epoch : 9, batch : 3339, Loss : 1.4914536222931929e-05
epoch : 9, batch : 3340, Loss : 4.233732397551648e-05
epoch : 9, batch : 3341, Loss : 0.00034122716169804335
epoch : 9, batch : 3342, Loss : 0.001809869660064578
epoch : 9, batch : 3343, Loss : 0.004655235446989536
epoch : 9, batch : 3344, Loss : 4.9657268391456455e-05
epoch : 9, batch : 3345, Loss : 4.394777351990342e-05
epoch : 9, batch : 3346, Loss : 1.1559190170373768e-05
epoch : 9, batch : 3347, Loss : 4.7464425733778626e-05
epoch : 9, batch : 3348, Loss : 0.00022856492432765663
epoch : 9, batch : 3349, Loss : 0.0011777940671890974
epoch : 9, batch : 3350, Loss : 2.0973764094378566e-06
epoch : 9, batch : 3351, Loss : 1.039278504322283e-05
epoch : 9, batch : 3352, Loss : 1.2554331760838977e-06
epoch : 9, batch : 3353, Loss : 0.00011122680734843016
epoch : 9, batch : 3354, Loss : 8.084670116659254e-05
epoch : 9, batch : 33

epoch : 9, batch : 3527, Loss : 0.001468383357860148
epoch : 9, batch : 3528, Loss : 0.0002529898192733526
epoch : 9, batch : 3529, Loss : 0.00026996369706466794
epoch : 9, batch : 3530, Loss : 0.00010093925084220245
epoch : 9, batch : 3531, Loss : 0.030185502022504807
epoch : 9, batch : 3532, Loss : 0.000801395159214735
epoch : 9, batch : 3533, Loss : 0.0021223926451057196
epoch : 9, batch : 3534, Loss : 3.198126796633005e-05
epoch : 9, batch : 3535, Loss : 0.00011408246791688725
epoch : 9, batch : 3536, Loss : 0.00033293836168013513
epoch : 9, batch : 3537, Loss : 7.246793393278494e-05
epoch : 9, batch : 3538, Loss : 0.0007129864534363151
epoch : 9, batch : 3539, Loss : 0.0010963666718453169
epoch : 9, batch : 3540, Loss : 0.00011331201676512137
epoch : 9, batch : 3541, Loss : 0.0014507368905469775
epoch : 9, batch : 3542, Loss : 0.027916792780160904
epoch : 9, batch : 3543, Loss : 0.00010955677862511948
epoch : 9, batch : 3544, Loss : 0.0036602516192942858
epoch : 9, batch : 3545, L

epoch : 9, batch : 3714, Loss : 4.1008126572705805e-05
epoch : 9, batch : 3715, Loss : 7.574879418825731e-05
epoch : 9, batch : 3716, Loss : 0.0037026223726570606
epoch : 9, batch : 3717, Loss : 0.005470912903547287
epoch : 9, batch : 3718, Loss : 0.00022343134332913905
epoch : 9, batch : 3719, Loss : 0.0007274594972841442
epoch : 9, batch : 3720, Loss : 0.0003427963238209486
epoch : 9, batch : 3721, Loss : 0.000630280701443553
epoch : 9, batch : 3722, Loss : 1.225163032358978e-05
epoch : 9, batch : 3723, Loss : 0.0036353450268507004
epoch : 9, batch : 3724, Loss : 7.222245039884001e-05
epoch : 9, batch : 3725, Loss : 5.5007560149533674e-05
epoch : 9, batch : 3726, Loss : 0.0010327256750315428
epoch : 9, batch : 3727, Loss : 5.9961275837849826e-05
epoch : 9, batch : 3728, Loss : 0.0035280112642794847
epoch : 9, batch : 3729, Loss : 5.2908333600498736e-05
epoch : 9, batch : 3730, Loss : 5.140819848747924e-05
epoch : 9, batch : 3731, Loss : 0.00045312324073165655
epoch : 9, batch : 3732,

epoch : 9, batch : 3928, Loss : 0.00010691197530832142
epoch : 9, batch : 3929, Loss : 0.0009627027902752161
epoch : 9, batch : 3930, Loss : 0.00010060250497190282
epoch : 9, batch : 3931, Loss : 0.00020688885706476867
epoch : 9, batch : 3932, Loss : 0.001175565179437399
epoch : 9, batch : 3933, Loss : 0.0001191729970742017
epoch : 9, batch : 3934, Loss : 1.570056519994978e-05
epoch : 9, batch : 3935, Loss : 0.00046534763532690704
epoch : 9, batch : 3936, Loss : 0.00030926690669730306
epoch : 9, batch : 3937, Loss : 0.0004521850205492228
epoch : 9, batch : 3938, Loss : 0.0003107923548668623
epoch : 9, batch : 3939, Loss : 0.0005819774814881384
epoch : 9, batch : 3940, Loss : 0.0007749805808998644
epoch : 9, batch : 3941, Loss : 0.02440071478486061
epoch : 9, batch : 3942, Loss : 6.213845335878432e-05
epoch : 9, batch : 3943, Loss : 1.4434563126997091e-05
epoch : 9, batch : 3944, Loss : 1.2654759302677121e-05
epoch : 9, batch : 3945, Loss : 0.0006927852518856525
epoch : 9, batch : 3946,

In [13]:
# 定义验证函数
def val(model, val_loader):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            outputs = model(inputs)
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
            pred = outputs.argmax(dim=1, keepdim=True)
            correct += pred.eq(labels.view_as(pred)).sum().item()
            
    print('Accuracy {}%'.format(100 * correct / len(val_loader.dataset)))

In [14]:
val(model, val_dataloader)

Accuracy 99.82093325374811%
